In [1]:
import pandas as pd
import numpy as np  
import matplotlib.pyplot as plt
import seaborn as sns
import regex as re
import nltk
import math
import os
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

1. Understanding the Dataset

In [2]:
df = pd.read_parquet(r"C:\Users\Dell\Downloads\shopping_queries_dataset_examples.parquet")

In [3]:
print(df.head())
print(df.info())
print(df.describe())
print(df.isnull().sum())

   example_id           query  query_id  product_id product_locale esci_label  \
0           0   revent 80 cfm         0  B000MOO21W             us          I   
1           1   revent 80 cfm         0  B07X3Y6B1V             us          E   
2           2   revent 80 cfm         0  B07WDM7MQQ             us          E   
3           3   revent 80 cfm         0  B07RH6Z8KW             us          E   
4           4   revent 80 cfm         0  B07QJ7WYFQ             us          E   

   small_version  large_version  split  
0              0              1  train  
1              0              1  train  
2              0              1  train  
3              0              1  train  
4              0              1  train  
<class 'pandas.core.frame.DataFrame'>
Index: 2621288 entries, 0 to 2621287
Data columns (total 9 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   example_id      int64 
 1   query           object
 2   query_id        int64 
 3   product_id     

In [4]:
df.shape

(2621288, 9)

In [5]:
df_product = pd.read_parquet(r"C:\Users\Dell\Downloads\shopping_queries_dataset_products.parquet")

In [6]:
print(df_product.head())
print(df_product.info())
print(df_product.describe())
print(df_product.isnull().sum())

   product_id                                      product_title  \
0  B079VKKJN7  11 Degrees de los Hombres Playera con Logo, Ne...   
1  B079Y9VRKS          Camiseta Eleven Degrees Core TS White (M)   
2  B07DP4LM9H  11 Degrees de los Hombres Core Pull Over Hoodi...   
3  B07G37B9HP          11 Degrees Poli Panel Track Pant XL Black   
4  B07LCTGDHY  11 Degrees Gorra Trucker Negro OSFA (Talla úni...   

                                 product_description  \
0  Esta playera con el logo de la marca Carrier d...   
1                                               None   
2  La sudadera con capucha Core Pull Over de 11 G...   
3                                               None   
4                                               None   

                                product_bullet_point product_brand  \
0  11 Degrees Negro Playera con logo\nA estrenar ...    11 Degrees   
1                                               None    11 Degrees   
2  11 Degrees Azul Core Pull Over Hoodie\nA 

In [7]:
df_product.shape

(1814924, 7)

2. Data Preprocessing 

In [8]:
df = df.merge(df_product, on="product_id", how="left")
df.head()

,example_id,query,query_id,product_id,product_locale_x,esci_label,small_version,large_version,split,product_title,product_description,product_bullet_point,product_brand,product_color,product_locale_y
0,0,revent 80 cfm,0,B000MOO21W,us,I,0,1,train,Panasonic FV-20VQ3 WhisperCeiling 190 CFM Ceil...,None,WhisperCeiling fans feature a totally enclosed...,Panasonic,White,us
1,1,revent 80 cfm,0,B07X3Y6B1V,us,E,0,1,train,Homewerks 7141-80 Bathroom Fan Integrated LED ...,None,OUTSTANDING PERFORMANCE: This Homewerk's bath ...,Homewerks,80 CFM,us
2,2,revent 80 cfm,0,B07WDM7MQQ,us,E,0,1,train,Homewerks 7140-80 Bathroom Fan Ceiling Mount E...,None,OUTSTANDING PERFORMANCE: This Homewerk's bath ...,Homewerks,White,us
3,3,revent 80 cfm,0,B07RH6Z8KW,us,E,0,1,train,Delta Electronics RAD80L BreezRadiance 80 CFM ...,This pre-owned or refurbished product has been...,Quiet operation at 1.5 sones\nBuilt-in thermos...,DELTA ELECTRONICS (AMERICAS) LTD.,White,us
4,4,revent 80 cfm,0,B07QJ7WYFQ,us,E,0,1,train,Panasonic FV-08VRE2 Ventilation Fan with Reces...,None,The design solution for Fan/light combinations...,Panasonic,White,us


In [9]:
df["product_text"] = (df["product_title"] + "-" + df["product_description"]).str.strip()

In [10]:
df.head(10)

,example_id,query,query_id,product_id,product_locale_x,esci_label,small_version,large_version,split,product_title,product_description,product_bullet_point,product_brand,product_color,product_locale_y,product_text
0,0,revent 80 cfm,0,B000MOO21W,us,I,0,1,train,Panasonic FV-20VQ3 WhisperCeiling 190 CFM Ceil...,None,WhisperCeiling fans feature a totally enclosed...,Panasonic,White,us,NaN
1,1,revent 80 cfm,0,B07X3Y6B1V,us,E,0,1,train,Homewerks 7141-80 Bathroom Fan Integrated LED ...,None,OUTSTANDING PERFORMANCE: This Homewerk's bath ...,Homewerks,80 CFM,us,NaN
2,2,revent 80 cfm,0,B07WDM7MQQ,us,E,0,1,train,Homewerks 7140-80 Bathroom Fan Ceiling Mount E...,None,OUTSTANDING PERFORMANCE: This Homewerk's bath ...,Homewerks,White,us,NaN
3,3,revent 80 cfm,0,B07RH6Z8KW,us,E,0,1,train,Delta Electronics RAD80L BreezRadiance 80 CFM ...,This pre-owned or refurbished product has been...,Quiet operation at 1.5 sones\nBuilt-in thermos...,DELTA ELECTRONICS (AMERICAS) LTD.,White,us,Delta Electronics RAD80L BreezRadiance 80 CFM ...
4,4,revent 80 cfm,0,B07QJ7WYFQ,us,E,0,1,train,Panasonic FV-08VRE2 Ventilation Fan with Reces...,None,The design solution for Fan/light combinations...,Panasonic,White,us,NaN
5,5,revent 80 cfm,0,B076Q7V5WX,us,E,0,1,train,Panasonic FV-0511VQ1 WhisperCeiling DC Ventila...,None,Installation: Features a 4-inch or 6-inch duct...,Panasonic,White,us,NaN
6,6,revent 80 cfm,0,B075ZBF9HG,us,E,0,1,train,Panasonic FV-0510VSL1 WhisperValue DC Ventilat...,None,Installation: Features a low profile can ideal...,Panasonic,White,us,NaN
7,7,revent 80 cfm,0,B06W2LB17J,us,E,0,1,train,Panasonic FV-0510VS1 WhisperValue DC Ventilati...,None,Installation: Features a low profile can ideal...,Panasonic,White,us,NaN
8,8,revent 80 cfm,0,B07JY1PQNT,us,E,0,1,train,Aero Pure ABF80 L5 W ABF80L5 Ceiling Mount 80 ...,None,"Quiet 0.3 Sones, 80 CFM fan with choice of thr...",Aero Pure,White,us,NaN
9,9,revent 80 cfm,0,B01MZIK0PI,us,E,0,1,train,Delta Electronics (Americas) Ltd. RAD80 Delta ...,None,Quiet operation at 1.5 Sones\nPrecision engine...,DELTA ELECTRONICS (AMERICAS) LTD.,With Heater,us,NaN


In [11]:
df['product_text']

0                                                        NaN
1                                                        NaN
2                                                        NaN
3          Delta Electronics RAD80L BreezRadiance 80 CFM ...
4                                                        NaN
                                 ...                        
2680359    薄い財布 abrAsus ブッテーロレザーエディション ブラック-2013年グッドデザイン賞...
2680360    アブラサス (abrAsus) 薄い財布 ブラック-グッドデザイン賞受賞。 <br /> 「...
2680361    Bellroy Hide & Seek Wallet - スリムなレザー製二つ折り財布、RF...
2680362    小さい小銭入れ abrAsus (アブラサス) ダークグリーン-<b>「コイン」「紙幣」「キ...
2680363    小さい財布 abrAsus アブラサス 最上級ブッテーロレザーエディション ブルー-手のひら...
Name: product_text, Length: 2680364, dtype: object

In [12]:
df['esci_label'].value_counts()

esci_label
E    1744103
S     588743
I     269833
C      77685
Name: count, dtype: int64

In [13]:
df['product_text'] = df['product_text'].str.lower()

In [14]:
df["product_text"] = df["product_text"].fillna(" ")

In [15]:
df.head(10)

,example_id,query,query_id,product_id,product_locale_x,esci_label,small_version,large_version,split,product_title,product_description,product_bullet_point,product_brand,product_color,product_locale_y,product_text
0,0,revent 80 cfm,0,B000MOO21W,us,I,0,1,train,Panasonic FV-20VQ3 WhisperCeiling 190 CFM Ceil...,None,WhisperCeiling fans feature a totally enclosed...,Panasonic,White,us,
1,1,revent 80 cfm,0,B07X3Y6B1V,us,E,0,1,train,Homewerks 7141-80 Bathroom Fan Integrated LED ...,None,OUTSTANDING PERFORMANCE: This Homewerk's bath ...,Homewerks,80 CFM,us,
2,2,revent 80 cfm,0,B07WDM7MQQ,us,E,0,1,train,Homewerks 7140-80 Bathroom Fan Ceiling Mount E...,None,OUTSTANDING PERFORMANCE: This Homewerk's bath ...,Homewerks,White,us,
3,3,revent 80 cfm,0,B07RH6Z8KW,us,E,0,1,train,Delta Electronics RAD80L BreezRadiance 80 CFM ...,This pre-owned or refurbished product has been...,Quiet operation at 1.5 sones\nBuilt-in thermos...,DELTA ELECTRONICS (AMERICAS) LTD.,White,us,delta electronics rad80l breezradiance 80 cfm ...
4,4,revent 80 cfm,0,B07QJ7WYFQ,us,E,0,1,train,Panasonic FV-08VRE2 Ventilation Fan with Reces...,None,The design solution for Fan/light combinations...,Panasonic,White,us,
5,5,revent 80 cfm,0,B076Q7V5WX,us,E,0,1,train,Panasonic FV-0511VQ1 WhisperCeiling DC Ventila...,None,Installation: Features a 4-inch or 6-inch duct...,Panasonic,White,us,
6,6,revent 80 cfm,0,B075ZBF9HG,us,E,0,1,train,Panasonic FV-0510VSL1 WhisperValue DC Ventilat...,None,Installation: Features a low profile can ideal...,Panasonic,White,us,
7,7,revent 80 cfm,0,B06W2LB17J,us,E,0,1,train,Panasonic FV-0510VS1 WhisperValue DC Ventilati...,None,Installation: Features a low profile can ideal...,Panasonic,White,us,
8,8,revent 80 cfm,0,B07JY1PQNT,us,E,0,1,train,Aero Pure ABF80 L5 W ABF80L5 Ceiling Mount 80 ...,None,"Quiet 0.3 Sones, 80 CFM fan with choice of thr...",Aero Pure,White,us,
9,9,revent 80 cfm,0,B01MZIK0PI,us,E,0,1,train,Delta Electronics (Americas) Ltd. RAD80 Delta ...,None,Quiet operation at 1.5 Sones\nPrecision engine...,DELTA ELECTRONICS (AMERICAS) LTD.,With Heater,us,


In [16]:
url_pattern = re.compile(r'https?://\S+')

def remove_urls(text):
    return url_pattern.sub('', text)

df["product_text"] = df["product_text"].apply(remove_urls)

In [17]:
df = df.replace(to_replace=r'[^\w\s]', value='', regex=True)

In [18]:
df = df.replace(to_replace=r'\d', value='', regex=True)

In [19]:
# Tokenization
nltk.download('punkt')
nltk.download('punkt_tab')

df['product_text'] = df['product_text'].apply(word_tokenize)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [20]:
# Stopword Removal
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))
df['product_text'] = df['product_text'].apply(lambda x: [word for word in x if word not in stop_words])

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [21]:
# Lemmatization 
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()

#Function for lemmatization
def lemmatize_text(text):
    if isinstance(text, list):  # convert list to string
        text = ' '.join(text)
    if not isinstance(text, str) or not text.strip():
        return ''
    return ' '.join(lemmatizer.lemmatize(w) for w in text.split())

df['lemmatized_text'] = df['product_text'].apply(lemmatize_text)

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [24]:
output_path = r"C:\Users\Dell\Documents\final_processed_dataframe.pkl"

# Save dataframe as compressed pickle file
df.to_pickle(output_path, compression="gzip")

print("DataFrame saved successfully as a compressed Pickle file!")

DataFrame saved successfully as a compressed Pickle file!
